# Preamble

This notebook presents the nitrate database compilation. It is centered around Codispoti et al.'s 2013 compilation of historical nitrate measurements in the Arctic Ocean.

All figures exported from this notebook are prefixed with `FIGURE_FN-COMP_`.

In [ ]:
%run imports.py
%load_ext autoreload
%autoreload 2

In [ ]:
# Define dimensions
dims = dict(
    reg_name=hv.Dimension('reg_name', label='Region'),
    ntr0=hv.Dimension('ntr0', label='Surface NO₃ conc.', range=(0,13)),
    doy=hv.Dimension('doy', label='Day of the year')
)

# Define individual datasets

- Arrigo et al 2017: CTD+nut+chla:  `databases/arrigo2017`
    - https://datadryad.org/resource/doi:10.5061/dryad.fm7b5
    - https://doi.org/10.5061/dryad.fm7b5
    
- Pierre Coupel's compilation (ArcticNet+others): `databases/arcticnet-nutrients-pierre/`
- Codispoti et al.'s compilation:  `databases/codispoti2013synthesis-1.1/` 

Common dataframe format:
- station < cast no, station,...
- date
    - year
    - month
    - day
- depth
    - p
- lon
- lat
- temperature
    - CT
- salinity
    - SA
- sigth
- nitrate

## Codispoti et al.

In [ ]:
def load_codispotietal():
    """
    Codispoti et al., 2013
    """
    renamedict = dict(NO3='nitrate', Sal='sal', 
                  T='temp', z='depth', Longitude='lon', 
                  Latitude='lat', Date='date', Station='station', Cruise='cruise'
                 )

    df = (pd.read_csv('/Users/doppler/database/codispoti2013synthesis-1.1/data/0-data/Codispoti_Arctic_Nutrients_Submission_11-11-2010.csv',
                      na_values=-999,
                      parse_dates=['Date']
                     )[['Date','Latitude','Longitude','NO3','z','T','Sal','Station','Cruise']]
           .rename(columns=renamedict)
           .dropna(subset=['nitrate'])
          )
    df = df.assign(station=lambda row: row.cruise+'+'+row.station.astype(str))
    df = df.drop(columns=['cruise'])
    df = df.groupby(['station', 'date', 'depth']).mean().reset_index()

    df['p'] = gsw.p_from_z(-df.depth, df.lat)
    df['SA'] = gsw.SA_from_SP(df.sal, df.p, df.lon, df.lat)
    df['CT'] = gsw.CT_from_pt(df.SA, df.temp)
    df['sigth'] = gsw.sigma0(df.SA, df.CT)

    df = df.assign(database='codispoti')
    return df  

## [include!!] Arrigo 2017, SUBICE

In [ ]:
def load_arrigoetal():
    df = pd.read_csv('/Users/doppler/database/arrigo2017/SUBICE_high_NO3_hy1.csv', 
                     na_values=-999, skiprows=9, header=[0,1],
                    ).replace(to_replace=-999, value=np.nan)
    df.columns = df.columns.droplevel(1)
    df = df.loc[df.BTLDPTH<=10].groupby('STNNBR').mean().reset_index()

    df.DATE = df.DATE.apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

    renamedict = dict(DATE='date',LATITUDE='latitude', LONGITUDE='longitude', NITRAT='nitrate', DEPTH='depth', STNNBR='station')

    df = df.rename(columns=renamedict)
    return df 

## [exclude?] Arcticnet etc (Pierre)

Define load_ routine

# Compile and postprocess data 

## Merge all nutrient databases

In [ ]:
dfl = [
    load_codispotietal(),
    load_arrigoetal(),
]

df = pd.concat(dfl, sort=False).reset_index()
df.to_pickle('../data/no3-compilation/tmp_compiled.pandas')

## station-wise depth interpolation [takes time]

In [ ]:
df = pd.read_pickle('../data/no3-compilation/tmp_compiled.pandas')

def groupwise_interp(df):
    if df.depth.min()>20:
        return None
    else:
        bins = np.arange(-1,301.1,2)
        labels = bins[:-1]+np.diff(bins)/2
        df.depth = pd.to_numeric(pd.cut(df.depth, bins=bins, labels=labels))
        df = (df
              .dropna(subset=['depth'])
              .groupby('depth').mean()
              .reindex(index=labels)
             )
        return df.interpolate(method='linear', limit_area='inside').fillna(method='bfill').drop(
            columns=['station', 'date', 'depth'], errors='ignore')
    
df = (df.groupby(['database', 'station', 'date'])
      .apply(groupwise_interp)
      .reset_index()
     )

df.to_pickle('../data/no3-compilation/tmp_compiled-interpolated.pandas')

## Derive per-profile quantities

In [ ]:
df = pd.read_pickle('../data/no3-compilation/tmp_compiled-interpolated.pandas')
df = df.loc[df.database.isin(['codispoti', 'arcticnet'])]

### MLD

In [ ]:
def find_ml (sigth, depth, delta_sigth_crit=0.1):
    """
    Find mixed layer with sfc. density+0.1 kg/m3 criterion.
    """
    index = np.where(sigth>np.nanmean(sigth.iloc[:5]) + delta_sigth_crit)[0]
    if len(index)>0:
        return depth.iloc[index[0]].astype(float)
    else:
        return np.nan

In [ ]:
def find_nitracline(no3, depth):
    no3sfc = no3.loc[depth<=10].mean()
    index = np.where(no3>no3sfc+1.)[0]
    if len(index)>0:
        return depth.iloc[index[0]].astype(float)
    else:
        return np.nan

df=df.merge(pd.DataFrame(
            df.groupby(df.station).apply(lambda g: find_ml(g.sigth, g.depth)),
            columns=['mld']),on='station')

### Nitracline

There are two relevant nitraclines:
1. `nc0`: Biological, when NO3 jumps over say 1uM
1. `nc`: Physical, when NO3 jumps over say sfc.NO3+1uM

In [ ]:
def find_nitracline0(no3, depth, no3crit=1.0):
    index = np.where(no3>=no3crit)[0]
    if len(index)>0:
        return depth.iloc[index[0]].astype(float)
    else:
        return np.nan
    
def find_nitracline(no3, depth):
    no3sfc = no3.loc[depth<=10].mean()
    index = np.where(no3>no3sfc+1.)[0]
    if len(index)>0:
        return depth.iloc[index[0]].astype(float)
    else:
        return np.nan
    
gb = df.groupby(df.station)
df = df.merge(pd.DataFrame(
    dict(nc0=gb.apply(lambda g: find_nitracline0(g.nitrate, g.depth, 1)),
         nc   =gb.apply(lambda g: find_nitracline(g.nitrate, g.depth))
        )),on='station')

### [unused] Nitrate vs density gradient strength

### Stratification 

Take one of the measures discussed earlier... (0-100m, deltasight, ...)

In [ ]:
sigth0 = df.loc[df.depth.between(35, 45)].groupby('station').mean().sigth
sigth_deep = df.loc[df.depth.between(95, 105)].groupby('station').mean().sigth

df = df.merge(pd.DataFrame(
    dict(delta_sigth=(sigth_deep-sigth0).values, station=sigth0.index.values)
))

In [ ]:
def buoyancy_freq(sigma, depth, from_depth, layer_thickness=70):
    """
    Calculate buoyancy frequency
    over depth range [from_depth, from_depth+layer_thickness].
    """
    nc_depth_range = (from_depth<=depth) & (depth<=from_depth+layer_thickness)
    if not np.any(nc_depth_range):
        return np.nan
    else:
        def slope(x, y, x_range):
            return np.polyfit(x.loc[x_range], y.loc[x_range].sort_values(), 1)[0]

        return 9.81/(1e3+np.mean(sigma)) * slope(depth, sigma, nc_depth_range)

gb = df.groupby(df.station)
df = df.merge(pd.DataFrame(
    dict(
        N2_30_100=gb.apply(lambda g: buoyancy_freq(g.sigth, g.depth, g.mld, 30))
    )),
    on='station')

### Save database

In [ ]:
dfp = df.copy()
df = (df
       .loc[df.depth==df.depth.min()]
#        [['date', 'lon', 'lat', 'station', 'nitrate', 'nc', 'nc0_1', 'nc0_2', 'mld', 'SA', 'CT', 
#          'sigth', 'no3sig']]#, 'no3sig_nc0_1', 'no3sig_nc0_2']]
       .rename(columns=dict(nitrate='ntr0'))
      )

dfp = dfp.merge(df[['station', 'ntr0']], how='outer')

In [ ]:
df.to_pickle('../data/no3-compilation/tmp_compiled-interpolated-derived-per-station.pandas')
dfp.to_pickle('../data/no3-compilation/tmp_compiled-interpolated-derived.pandas')

## Add regions

Based off Peralta-Ferriz & Woodgate and Codispoti et al.

### Define regions

In [ ]:
d = [
    ('Chukchi Sea', smoothen(box(-180, 68, -155, 76))), 
    ('Southern Beaufort', smoothen(box(-155, 68, -115, 72))), 
    ('Canada Basin', smoothen(box(-155, 72, -130, 84))), 
    ('Makarov Basin', smoothen(box(-180, 83.5, -50, 90)).union(smoothen(box(140, 78, 180, 90)))), 
    ('Eurasian Basin', smoothen(box(-30, 82, 140, 90).union(box(110, 78, 140, 82)))), 
    ('Barents Sea', smoothen(box(15, 75, 60, 80).union(box(15, 70.5, 55, 75)))),
    ('Baffin Bay', smoothen(box(-65, 66, -45, 78))),
    ('Canadian Archipelago', smoothen(box(-110, 66, -65, 80))), #.union(box(-100, 78, -50, 82))))
    ('Fram Strait (East)', smoothen(box(-5, 75, 15, 81)))
]

names, geo = zip(*d)

regions = gpd.GeoDataFrame(dict(reg_name=list(names)), geometry=list(geo))
regions.crs = from_epsg(4326)

# regions = regions.to_crs(from_epsg(3413))

regions['reg_idx'] = range(len(d))

regions.to_file('../data/regions/arctic-regions.shp')

... and visualize them:

In [ ]:
regions = gpd.read_file('../data/regions/arctic-regions.shp')
options_bk = [
    opts.Overlay(legend_position='left'),
    opts.Polygons(projection=ccrs.NorthPolarStereo(), cmap='Category10', tools=['hover'], 
                  show_legend=True, 
                  width=500, aspect='equal',
                  line_color=None, alpha=.7,
                 ),
    opts.NdOverlay(show_legend=True),
]
add_backend_to_opts(options_bk, 'bokeh')
options_mpl = translate_options(options_bk, bokeh2mpl, override={'Polygons': {'edgecolor': 'none'}})

poly = gv.Polygons(regions, kdims=['Longitude', 'Latitude'], vdims=['reg_name', 'reg_idx'])
l = poly *gf.land * gf.coastline * graticules * isobath2000
l = l.opts(*options_bk)

fname = '../nb_fig/FIGURE_NO3-COMP_regions.png'
hv.save(l.opts(toolbar=None, clone=True), fname, fmt='png')
hv.save(l.opts(toolbar=None), fname, fmt='html')

## Add ancillary vars and regionalize dataframe

In [ ]:
df = pd.read_pickle('../data/no3-compilation/tmp_compiled-interpolated-derived-per-station.pandas')
dfp = pd.read_pickle('../data/no3-compilation/tmp_compiled-interpolated-derived.pandas')

def season(timestamp):
    mm = timestamp.month
    if mm<=4:
        return 'winter'
    elif mm>=7 and mm<=9:
        return 'summer'
    
def add_info(df):
    df = (df
          .assign(doy=df.date.dt.dayofyear)
          .assign(month=df.date.dt.month)
          .assign(deltanc=df.nc-df.mld)
          .assign(deltanc0=df.nc0-df.mld)
          .assign(year=df.date.dt.year)
     )
    df['season'] = df.date.apply(season)
    return df

df = add_info(df)

gdf = df_to_gdf(df)
gdf = (gpd.sjoin(gdf, regions, op='within', how='left')
        .reset_index()
        .drop(columns=['index_right', 'index_left', 'index'], errors='ignore')
       )
df = pd.DataFrame(gdf).drop(columns=['geometry'])

dfp = dfp.merge(df, how='outer')
dfp = add_info(dfp)

df.to_csv('../data/no3-compilation/database-per-stn.csv', index=False)
dfp.to_csv('../data/no3-compilation/database.csv', index=False)
df.to_feather('../data/no3-compilation/database-per-stn.feather')
dfp.to_feather('../data/no3-compilation/database.feather')